In [ ]:
!pip install rasterio

In [1]:
import os
import rasterio
import numpy as np
import geopandas as gpd
from shapely.geometry import mapping
from rasterio.mask import mask
from rasterio.windows import Window
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
tif_path = "./prodes_amazonia_legal_2023.tif"
mask_path = "./desmatamento_2023_mask.tif"
ano_interesse = 23

with rasterio.open(tif_path) as src:
    profile = src.profile
    profile.update(
        dtype=rasterio.uint16,
        count=1,
        compress='lzw',
        nodata=255
    )

    with rasterio.open(mask_path, 'w', **profile) as dst:
        width = src.width
        height = src.height
        bloco_tam = 1024

        for y in range(0, height, bloco_tam):
            for x in range(0, width, bloco_tam):
                w = min(bloco_tam, width - x)
                h = min(bloco_tam, height - y)
                window = Window(x, y, w, h)

                bloco = src.read(1, window=window)
                mask_ano = np.where(bloco == ano_interesse, 1, 0).astype(np.uint16)

                dst.write(mask_ano, 1, window=window)


In [ ]:
shapefile = "./ibge_malha_municipal/BR_Municipios_2024.shp"
raster_path = "./desmatamento_2023_mask.tif"
raster_reproj = "./desmatamento_2023_reproj_5880.tif"

dst_crs = "EPSG:5880"
with rasterio.open(raster_path) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)

    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    with rasterio.open(raster_reproj, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest)


Raster reprojetado para EPSG:5880!


In [ ]:
import rasterio
import numpy as np

raster_reproj = "./desmatamento_2023_reproj_5880.tif"

with rasterio.open(raster_reproj) as src:
    print("CRS:", src.crs)
    print("Dimensões (altura, largura):", src.height, src.width)
    print("Número de bandas:", src.count)
    print("Resolução (m/pixel):", src.res)
    print("Extensão (bounds):", src.bounds)

    raster_data = src.read(1)

    unique_vals = np.unique(raster_data)
    print("Valores únicos:", unique_vals[:20], "...")


CRS: EPSG:5880
Dimensões (altura, largura): 91608 112276
Número de bandas: 1
Resolução (m/pixel): (29.783034124460546, 29.783034124460546)
Extensão (bounds): BoundingBox(left=2774617.9887131862, bottom=7890145.21942072, right=6118537.928071119, top=10618509.409494301)


In [ ]:
shapefile = "./ibge_malha_municipal/BR_Municipios_2024.shp"
raster_reproj = "./desmatamento_2023_reproj_5880.tif"

estados_amazonia_legal = ['AC', 'AP', 'AM', 'MA', 'MT', 'PA', 'RO', 'RR', 'TO']
gdf = gpd.read_file(shapefile)
gdf = gdf[gdf["SIGLA_UF"].isin(estados_amazonia_legal)].copy()
gdf = gdf.to_crs("EPSG:5880")

with rasterio.open(raster_reproj) as src:
    pixel_area_m2 = abs(src.res[0] * src.res[1])
    pixel_area_ha = pixel_area_m2 / 10_000

    resultados = []

    print("Calculando desmatamento por município...")
    for idx, row in tqdm(gdf.iterrows(), total=gdf.shape[0]):
        try:
            out_image, out_transform = mask(src, [mapping(row.geometry)], crop=True)
            data = out_image[0]

            mask_valid = data != src.nodata

            desmatado = (data == 1) & mask_valid

            n_pixels_desmatado = np.sum(desmatado)

            area_desmatada_ha = n_pixels_desmatado * pixel_area_ha

            resultados.append({
                "id_municipio": row["CD_MUN"],
                "nome_municipio": row["NM_MUN"],
                "area_desmatada_ha": area_desmatada_ha
            })
        except Exception as e:
            print(f"Erro no município {row['NM_MUN']}: {e}")
            continue

df_resultado = pd.DataFrame(resultados)
df_resultado.to_csv("./area_desmatada_por_municipio.csv", index=False)

Calculando desmatamento por município...


  4%|▍         | 36/809 [01:57<2:05:25,  9.74s/it]

Erro no município Afonso Cunha: Input shapes do not overlap raster.


  6%|▋         | 52/809 [02:58<1:04:24,  5.10s/it]

Erro no município Água Doce do Maranhão: Input shapes do not overlap raster.


...

Erro no município Santa Quitéria do Maranhão: Input shapes do not overlap raster.


 98%|█████████▊| 790/809 [49:31<00:34,  1.81s/it]

Erro no município Araioses: Input shapes do not overlap raster.


100%|██████████| 809/809 [51:17<00:00,  3.80s/it]


✅ Cálculo finalizado! Resultado salvo em area_desmatada_por_municipio.csv


In [ ]:
df = pd.read_csv("./area_desmatada_por_municipio.csv")

In [7]:
df.query("id_municipio == 1500602")

,id_municipio,nome_municipio,area_desmatada_ha
559,1500602,Altamira,31761.940462
